In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import gpxSense as gpx
import matplotlib.pyplot as plt

2025-02-09 13:29:58.192635: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-09 13:29:58.202921: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 13:29:58.305107: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-09 13:29:58.392773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739104198.480256    7234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739104198.50

In [3]:
def huber_loss(y_true, y_pred, clip_delta=1.0):
	error = y_true - y_pred
	cond  = tf.keras.backend.abs(error) < clip_delta

	squared_loss = 0.5 * tf.keras.backend.square(error)
	linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)

	return tf.where(cond, squared_loss, linear_loss)

def huber_loss_mean(y_true, y_pred, clip_delta=1.0):
	return tf.keras.backend.mean(huber_loss(y_true, y_pred, clip_delta))

def combined_loss(alpha):
    def loss(y_true, y_pred):
        mse = tf.keras.backend.mean(tf.keras.backend.square(y_true - y_pred))
        mae = tf.keras.backend.mean(tf.keras.backend.abs(y_true - y_pred))
        return alpha * mse + (1 - alpha) * mae
    return loss

def from_df_to_tfds(df, y, shuffle=True, batch_size=32):
	ds = tf.data.Dataset.from_tensor_slices((df.values, y.values))
	if shuffle:
		ds = ds.shuffle(buffer_size=len(df))
	ds = ds.batch(batch_size)
	return ds

def performance(model, df_train, df_norm_test, y_test):
	evaluations = model.evaluate(from_df_to_tfds(df_norm_test, y_test, shuffle=False, batch_size=1))
	mean_pred = model.predict(from_df_to_tfds(df_norm_test, y_test, shuffle=False, batch_size=1)).mean() * (df_train.max()['hr'] - df_train.min()['hr']) + df_train.min()['hr']
	real_mean = y_test.mean() * (df_train.max()['hr'] - df_train.min()['hr']) + df_train.min()['hr']
	print("Mean prediction:\n", f"{mean_pred:.2f} bpm")
	print("Real mean:\n", f"{real_mean:.2f} bpm")
	print(f"Error in the prediction:\n" f"{abs(mean_pred - real_mean) / real_mean * 100 : .2f}" + f"%")
	print("Metrics:\n", evaluations)
	loss = (evaluations[0] * (df_train['hr'].max() - df_train['hr'].min()))
	mae_loss = (evaluations[1] * (df_train['hr'].max() - df_train['hr'].min()))
	print("Loss:\n", f"{loss:.2f} bpm")
	print("MAE loss:\n", f"{mae_loss:.2f} bpm")
	print(f"Error in the prediction:\n" f"{(evaluations[1]) * 100 : .2f}" + f"%")
	return evaluations

In [4]:
df1 = gpx.gpxAnalyseClass("Data/31-05-2024.gpx").df
df2 = gpx.gpxAnalyseClass("Data/05-06-2024.gpx").df
df3 = gpx.gpxAnalyseClass("Data/18-06-2024.gpx").df
df4 = gpx.gpxAnalyseClass("Data/11-07-2024.gpx").df
df5 = gpx.gpxAnalyseClass("Data/07-12-2024.gpx").df
df6 = gpx.gpxAnalyseClass("Data/09-01-2025.gpx").df
df7 = gpx.gpxAnalyseClass("Data/23-01-2025.gpx").df
df8 = gpx.gpxAnalyseClass("Data/29-01-2025.gpx").df
df9 = gpx.gpxAnalyseClass("Data/31-01-2025.gpx").df
df10 = gpx.gpxAnalyseClass("Data/04-02-2025.gpx").df
df11 = gpx.gpxAnalyseClass("Data/07-02-2025.gpx").df

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df11], axis=0)
df = df.drop(['lat', 'lon'], axis=1)
df = df.select_dtypes(include=['number'])
print(df.mean())
print(len(df))
df_norm = (df - df.min()) / (df.max() - df.min())

ele           38.432323
hr           153.724518
cad          178.290960
dist        4099.041451
time_acc      25.962660
pace           6.197837
dtype: float64
6073


In [7]:
gpxTest = gpx.LinearPredictor("Data/TestingData1.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('DNNmodelPruebas.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
performance(DNNmodel, df, df_norm_test, y_test)

440/440 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - loss: 0.0038 - mae: 0.0596
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step
Mean prediction:
 151.45 bpm
Real mean:
 152.44 bpm
Error in the prediction:
 0.65%
Metrics:
 [0.0035008627455681562, 0.060681745409965515]
Loss:
 0.41 bpm
MAE loss:
 7.16 bpm
Error in the prediction:
 6.07%


[0.0035008627455681562, 0.060681745409965515]

In [8]:
gpxTest = gpx.LinearPredictor("Data/TestingData2.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('DNNmodelPruebas.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
performance(DNNmodel, df, df_norm_test, y_test)

530/530 ━━━━━━━━━━━━━━━━━━━━ 1s 806us/step - loss: 0.0036 - mae: 0.0587
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step
Mean prediction:
 154.17 bpm
Real mean:
 149.11 bpm
Error in the prediction:
 3.40%
Metrics:
 [0.00473410077393055, 0.06913475692272186]
Loss:
 0.56 bpm
MAE loss:
 8.16 bpm
Error in the prediction:
 6.91%


[0.00473410077393055, 0.06913475692272186]

In [17]:
print("--------Test 1--------")
gpxTest = gpx.LinearPredictor("Data/08-03-2024_Test1.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('DNNHBRMEANBS32LN00065LR.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

print("--------Test 2--------")
gpxTest = gpx.LinearPredictor("Data/25-04-2024_Test2.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('DNNHBRMEANBS32LN00065LR.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

print("--------Test 3--------")
gpxTest = gpx.LinearPredictor("Data/25-01-2024_Test3.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('DNNHBRMEANBS32LN00065LR.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

print("--------Test 4--------")
gpxTest = gpx.LinearPredictor("Data/03-02-2024_Test4.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('DNNHBRMEANBS32LN00065LR.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

--------Test 1--------
530/530 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0036 - mae: 0.0578
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step
Mean prediction:
 153.07 bpm
Real mean:
 149.11 bpm
Error in the prediction:
 2.65%
Metrics:
 [0.004703052807599306, 0.0649682879447937]
Loss:
 0.55 bpm
MAE loss:
 7.67 bpm
Error in the prediction:
 6.50%
--------Test 2--------
495/495 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0075 - mae: 0.1032
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 849us/step
Mean prediction:
 152.57 bpm
Real mean:
 143.21 bpm
Error in the prediction:
 6.54%
Metrics:
 [0.007271388079971075, 0.0953764021396637]
Loss:
 0.86 bpm
MAE loss:
 11.25 bpm
Error in the prediction:
 9.54%
--------Test 3--------
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0049 - mae: 0.0805
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Mean prediction:
 160.91 bpm
Real mean:
 160.21 bpm
Error in the prediction:
 0.44%
Metrics:
 [0.004223781172186136, 0.07143956422805786]
Loss:
 0.50 bpm
MAE loss:
 8.43 bpm
Error

In [19]:
print("--------Test 1--------")
gpxTest = gpx.LinearPredictor("Data/08-03-2024_Test1.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('best_model_1.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

print("--------Test 2--------")
gpxTest = gpx.LinearPredictor("Data/25-04-2024_Test2.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('best_model_1.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

print("--------Test 3--------")
gpxTest = gpx.LinearPredictor("Data/25-01-2024_Test3.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('best_model_1.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

print("--------Test 4--------")
gpxTest = gpx.LinearPredictor("Data/03-02-2024_Test4.gpx")
df_test = gpxTest.df
df_norm_test = df_test.drop(['lat', 'lon'], axis=1)
df_norm_test = df_norm_test.select_dtypes(include=['number'])
df_norm_test = (df_norm_test - df.min()) / (df.max() - df.min())
y_test = df_norm_test.pop('hr')
DNNmodel = tf.keras.models.load_model('best_model_1.keras', custom_objects={'huber_loss_mean': huber_loss_mean})
eval = performance(DNNmodel, df, df_norm_test, y_test)

--------Test 1--------
530/530 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0038 - mae: 0.0534
530/530 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step
Mean prediction:
 152.44 bpm
Real mean:
 149.11 bpm
Error in the prediction:
 2.24%
Metrics:
 [0.004673334304243326, 0.06025400385260582]
Loss:
 0.55 bpm
MAE loss:
 7.11 bpm
Error in the prediction:
 6.03%
--------Test 2--------
495/495 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0082 - mae: 0.1022
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step
Mean prediction:
 152.43 bpm
Real mean:
 143.21 bpm
Error in the prediction:
 6.44%
Metrics:
 [0.007953757420182228, 0.09495022892951965]
Loss:
 0.94 bpm
MAE loss:
 11.20 bpm
Error in the prediction:
 9.50%
--------Test 3--------
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0054 - mae: 0.0781
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 1s 860us/step
Mean prediction:
 159.86 bpm
Real mean:
 160.21 bpm
Error in the prediction:
 0.22%
Metrics:
 [0.004314905032515526, 0.0626075491309166]
Loss:
 0.51 bpm
MAE loss:
 7.39 bpm
Er